In [2]:
import pandas as pd
with open('EmoWikipediaTalkPages.csv') as csv_file:
    conversations = pd.read_csv(csv_file)

In [3]:
import numpy as np
from numpy import array
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
    # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [71]:
steps = 3
eval_name = "Plutchik"

In [52]:
from sklearn.metrics import precision_score, f1_score, accuracy_score, roc_auc_score, recall_score
from darts.metrics import ope, mae, mse, mape, mase
import json
from statistics import mean

def results_pool(actual_list, actual_ts, future_ts, train_ts):
    predicted = [x.values()[0][0] for x in future_ts]
    verdict_true = [x[-1]>x[-2] for x in actual_list]
    verdict_predicted = [a>x[-2] for a,x in zip(predicted, actual_list)]
    precision = precision_score(verdict_true, verdict_predicted)
    f1 = f1_score(verdict_true, verdict_predicted)
    accuracy = accuracy_score(verdict_true, verdict_predicted)
    auc = roc_auc_score(verdict_true, verdict_predicted)
    #recall = recall_score(verdict_true, verdict_predicted)
    mse_err = mse(actual_ts, future_ts)
    #ope_err = ope(actual_ts, future_ts)
    mae_err = mae(actual_ts, future_ts)
    mape_err = mape(actual_ts, future_ts)
    #mase_err = mase(actual_ts, future_ts, train_ts)
    results = {'mse':mean(mse_err), 
               #'ope':mean(ope_err), 
               'mae':mean(mae_err), 
               'mape':mean(mape_err),
               #'mase':mean(mase_err),
               #'precision':precision, 
               #'f1':f1, 
               'accuracy':accuracy, 
               #'auc':auc, 
               #'recall':recall
              }
    with open(f'Results-SotA/NBeats-{steps}-{eval_name}.json', 'w') as file:
        json.dump(results, file)

In [53]:
levels={
    "boredom": 1,
    "disgust": 2,
    "loathing": 3,
    "contempt": 2,
    "rage": 3,
    "anger": 2,
    "annoyance": 1,
    "aggresiveness": 2,
    "interest": 1,
    "anticipation": 2,
    "vigilance": 3,
    "optimism": 2,
    "ecstasy": 3,
    "joy": 2,
    "serenity": 1,
    "love": 2,
    "admiration": 3,
    "trust": 2,
    "acceptance": 1,
    "submission": 2,
    "terror": 3,
    "fear": 2,
    "apprehension": 1,
    "awe": 2,
    "amazement": 3,
    "surprise": 2,
    "distraction": 1,
    "disapproval": 2,
    "grief": 3,
    "sadness": 2,
    "pensiveness": 1,
    "remorse": 2,
    "none": 0
}

In [72]:
conv_list = []
ids = list(set(conversations["conv_id"]))
for i in ids:
    conv_list.append({"conversation_id":i, "utterances":[]})
for index, row in conversations.iterrows():
    for i in conv_list:
        if row["conv_id"]==i["conversation_id"]:
            emo1 = levels[row["plutchik_emotion1"]]
            emo2 = levels[row["plutchik_emotion2"]]
            emo3 = levels[row["plutchik_emotion3"]]
            #i["utterances"].append(round((emo1+emo2+emo3)/3))
            i["utterances"].append((emo1+emo2+emo3)/3)

In [73]:
import random
def train_val_test_split(data, percent1, percent2):
    split_place1 = int(percent1*len(data))
    split_place2 = int((percent1+percent2)*len(data))
    random.shuffle(data)
    return data[:split_place1], data[split_place1:split_place2], data[split_place2:]

In [74]:
values_series = [i['utterances'] for i in conv_list if (0 not in i['utterances']) and (len(i['utterances'])>=steps+1)]
values_series

[[1.0,
  1.6666666666666667,
  1.6666666666666667,
  1.6666666666666667,
  2.0,
  2.0,
  1.3333333333333333,
  2.0,
  2.6666666666666665],
 [2.0,
  2.0,
  1.0,
  1.3333333333333333,
  1.0,
  2.0,
  2.0,
  2.0,
  1.6666666666666667,
  2.3333333333333335,
  1.6666666666666667,
  1.0],
 [1.3333333333333333,
  2.0,
  2.0,
  2.6666666666666665,
  1.6666666666666667,
  2.6666666666666665],
 [0.6666666666666666,
  2.0,
  2.0,
  1.3333333333333333,
  1.3333333333333333,
  0.6666666666666666],
 [1.0,
  2.0,
  1.6666666666666667,
  1.6666666666666667,
  0.6666666666666666,
  1.0,
  1.3333333333333333,
  1.0,
  0.3333333333333333,
  0.6666666666666666,
  1.6666666666666667,
  2.0,
  1.3333333333333333,
  1.3333333333333333,
  1.3333333333333333,
  1.3333333333333333,
  1.3333333333333333,
  0.6666666666666666],
 [0.6666666666666666,
  2.3333333333333335,
  2.0,
  1.6666666666666667,
  1.0,
  1.6666666666666667],
 [1.3333333333333333,
  0.3333333333333333,
  1.0,
  1.0,
  1.6666666666666667,
  1.3

In [75]:
len(values_series)

33

In [60]:
import numpy as np
np.mean([len(x) for x in values_series])

4.116788321167883

In [61]:
train_list_avg, val_list_avg, test_list_avg = train_val_test_split(values_series, 0.7, 0.15)

In [62]:
train_array_avg = [array(x) for x in train_list_avg]
train_avg = [x.reshape((x.shape[0], 1, 1)) for x in train_array_avg]

In [63]:
len(train_avg)

95

In [64]:
val_array_avg = [array(x) for x in val_list_avg]
val_avg = [x.reshape((x.shape[0], 1, 1)) for x in val_array_avg]

In [65]:
test_array_avg = [array(x) for x in test_list_avg]
test_avg = [x.reshape((x.shape[0], 1, 1)) for x in test_array_avg]

In [42]:
from darts.models import NBEATSModel
from darts import TimeSeries

In [66]:
test_avg_ts = [TimeSeries.from_values(x) for x in test_avg]

In [67]:
fit_series_avg = [TimeSeries.from_values(x) for x in train_avg]
val_series_avg = [TimeSeries.from_values(x) for x in val_avg]
predict_series_avg = [x[:-1] for x in test_avg_ts]
y_test_avg_ts = [x[-1] for x in test_avg_ts]

In [68]:
model_one_step_avg = NBEATSModel(input_chunk_length=steps, output_chunk_length=1)
model_one_step_avg.fit(fit_series_avg, val_series=val_series_avg, verbose = True)

[2022-03-29 01:49:25,152] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 1520 samples.
[2022-03-29 01:49:25,152] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 1520 samples.
[2022-03-29 01:49:25,221] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 64-bits; casting model to float64.
[2022-03-29 01:49:25,221] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 64-bits; casting model to float64.


  0%|          | 0/100 [00:00<?, ?it/s]

In [69]:
future_avg = model_one_step_avg.predict(n=1, series = predict_series_avg)

In [70]:
results_pool(test_list_avg, y_test_avg_ts, future_avg, predict_series_avg)

In [33]:
future_avg

[<TimeSeries (DataArray) (time: 1, component: 1, sample: 1)>
 array([[[1.36747712]]])
 Coordinates:
   * time       (time) int64 2
   * component  (component) object '0'
 Dimensions without coordinates: sample,
 <TimeSeries (DataArray) (time: 1, component: 1, sample: 1)>
 array([[[1.15507836]]])
 Coordinates:
   * time       (time) int64 2
   * component  (component) object '0'
 Dimensions without coordinates: sample,
 <TimeSeries (DataArray) (time: 1, component: 1, sample: 1)>
 array([[[1.36747712]]])
 Coordinates:
   * time       (time) int64 2
   * component  (component) object '0'
 Dimensions without coordinates: sample,
 <TimeSeries (DataArray) (time: 1, component: 1, sample: 1)>
 array([[[1.55899495]]])
 Coordinates:
   * time       (time) int64 5
   * component  (component) object '0'
 Dimensions without coordinates: sample,
 <TimeSeries (DataArray) (time: 1, component: 1, sample: 1)>
 array([[[1.36747712]]])
 Coordinates:
   * time       (time) int64 5
   * component  (componen